In [1]:
from gym import Env
from gym.spaces import Discrete, Box

import numpy as np
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

from py4j.java_gateway import JavaGateway


In [2]:
# Conecta no servidor de treinamento

gateway = JavaGateway()
minitruco_java = gateway.entry_point


In [3]:
# Env customizado do gym que encapsupla o servidor

class MinitrucoEnv(Env):
    def __init__(self):
        # Ações que podemos tomar: jogar uma carta de valor 0 até uma carta de valor 14
        self.action_space = Discrete(15)

        # Espaço de observação: os 3 primeiros elementos são as cartas em mãos,
        # o restante são os valores das cartas na mesa, pontuações, etc.
        # Detalhes em SituacaoJogo.java#ranges e #toObservacao
        low, high = np.array(minitruco_java.ranges()).transpose()
        self.observation_space = Box(low=np.float32(np.array(low)), high=np.float32(np.array(high)))

        self.episodio = None
        self.state = None

    def _get_obs(self):
        values = self.episodio.estado().split()
        return [float(val) for val in values]
        # return np.array(float_values[24:27], dtype=np.float32)

    def step(self, action):
        action = int(action)
        last_observation = self.state
        indice_carta = -1
        for i in range(3):
            if last_observation[i] == action:
                indice_carta = i
                break
        if indice_carta == -1:
            # Jogada inválida (não deve acontecer no Keras por causa do masking, mas
            # se rolar por conta do teste inicial, só ignora e mantém no mesmo estado)
            return last_observation, 0, False, {}

        self.episodio.executa(indice_carta)
        estado_str = self.episodio.estado()
        if estado_str == "EQUIPE 1 VENCEU":
            return last_observation, 1.0, True, {}
        if estado_str == "EQUIPE 2 VENCEU":
            return last_observation, -1.0, True, {}

        self.state = self._get_obs()
        ganhoPontosEquipe1 = self.state[13] - last_observation[13]
        ganhoPontosEquipe2 = self.state[14] - last_observation[14]

        # Dá 1 ponto de recompensa se a equipe 1 ganhou pontos e -1 se a equipe 2 ganhou pontos
        reward = ganhoPontosEquipe1 - ganhoPontosEquipe2
        terminated = False

        return self.state, reward, terminated, {}

    def render(self):
        pass

    def reset(self, seed=None, options=None):
        if self.episodio is not None:
            self.episodio.finaliza()

#         super().reset(seed=seed) # required by check_env

        self.episodio = minitruco_java.novoEpisodio()
        self.state = self._get_obs()

        return self.state


env = MinitrucoEnv()


In [4]:
# Roda alguns episódios com política "jogue uma carta aleatoriamente"

import time

episodes = 50
total_reward = 0
for episode in range(1, episodes+1):
    state = env.reset()
    terminated = False
    score = 0
    start = time.time()

    while not terminated:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, terminated, info = env.step(action)
        score+=reward
    total_reward += score
    print('Episode:{} Score:{} Time:{}s'.format(episode, score, time.time() - start))
print("Average reward per episode {}".format(total_reward/episodes))


Episode:1 Score:-1.0 Time:0.5723845958709717s
Episode:2 Score:-1.0 Time:0.46492505073547363s
Episode:3 Score:-13.0 Time:0.37071990966796875s
Episode:4 Score:-1.0 Time:0.3986818790435791s
Episode:5 Score:1.0 Time:0.5000889301300049s
Episode:6 Score:-1.0 Time:0.40629100799560547s
Episode:7 Score:1.0 Time:0.3818390369415283s
Episode:8 Score:-1.0 Time:0.3865320682525635s
Episode:9 Score:1.0 Time:0.4010961055755615s
Episode:10 Score:1.0 Time:0.43812108039855957s
Episode:11 Score:-1.0 Time:0.42306089401245117s
Episode:12 Score:1.0 Time:0.37663888931274414s
Episode:13 Score:-1.0 Time:0.421950101852417s
Episode:14 Score:-1.0 Time:0.2702200412750244s
Episode:15 Score:-1.0 Time:0.3798191547393799s
Episode:16 Score:1.0 Time:0.42183685302734375s
Episode:17 Score:-1.0 Time:0.4342000484466553s
Episode:18 Score:-15.0 Time:0.42229723930358887s
Episode:19 Score:-1.0 Time:0.39779019355773926s
Episode:20 Score:-1.0 Time:0.43346214294433594s
Episode:21 Score:-1.0 Time:0.4246330261230469s
Episode:22 Score:

In [8]:
states = env.observation_space.shape
actions = env.action_space.n

def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states[0])))
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

model = build_model(states, actions)

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 17)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                432       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 15)                375       
Total params: 1,407
Trainable params: 1,407
Non-trainable params: 0
_________________________________________________________________


In [6]:
class MaskingDQNAgent(DQNAgent):
    def __init__(self, *args, **kwargs):
        super(MaskingDQNAgent, self).__init__(*args, **kwargs)

    def compute_q_values(self, state):
        q_values = super().compute_q_values(state)
        mask = np.array([1 if self._is_action_valid(state, action) else -np.inf for action in range(self.nb_actions)])
        masked_q_values = q_values + mask
        return masked_q_values

    def _is_action_valid(self, state, action):
        return state[0][0] == action or state[0][1] == action or state[0][2] == action

def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = MaskingDQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=1000, target_model_update=1e-2)
    return dqn

dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
# Original era 50K steps
dqn.fit(env, nb_steps=50_000, visualize=False, verbose=1)


2023-11-24 19:28:54.783377: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 19:28:54.798993: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc8d5d829c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-11-24 19:28:54.799009: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
10000/10000 [==============================] - 143s 14ms/step - reward: -0.0205
211 episodes - episode_reward: -0.972 [-14.000, 1.000] - loss: 58.016 - mae: 63.641 - mean_q: 78.649

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 172s 17ms/step - reward: -0.0083
212 episodes - episode_reward: -0.392 [-11.000, 1.000] - loss: 3544.656 - mae: 498.051 - mean_q: 619.572

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 200s 20ms/step - reward: -0.0171
209 episodes - episode_reward: -0.794 [-14.000, 1.000] - loss: 3188717.250 - mae: 10568.498 - mean_q: 13438.499

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 220s 22ms/step - reward: -0.0185
207 episodes - episode_reward: -0.918 [-15.000, 1.000] - loss: 292090052608.0

In [7]:
scores = dqn.test(env, nb_episodes=50, visualize=False)
print("Average reward per episode: {}".format(np.mean(scores.history['episode_reward'])))


Testing for 50 episodes ...
Episode 1: reward: 1.000, steps: 48
Episode 2: reward: -6.000, steps: 53
Episode 3: reward: 1.000, steps: 50
Episode 4: reward: -1.000, steps: 45
Episode 5: reward: 1.000, steps: 43
Episode 6: reward: 1.000, steps: 47
Episode 7: reward: -1.000, steps: 43
Episode 8: reward: -1.000, steps: 48
Episode 9: reward: -1.000, steps: 51
Episode 10: reward: -9.000, steps: 47
Episode 11: reward: 1.000, steps: 56
Episode 12: reward: 1.000, steps: 51
Episode 13: reward: 1.000, steps: 50
Episode 14: reward: 1.000, steps: 52
Episode 15: reward: 1.000, steps: 44
Episode 16: reward: -8.000, steps: 53
Episode 17: reward: -1.000, steps: 44
Episode 18: reward: -1.000, steps: 48
Episode 19: reward: 1.000, steps: 48
Episode 20: reward: -1.000, steps: 53
Episode 21: reward: -1.000, steps: 50
Episode 22: reward: -6.000, steps: 47
Episode 23: reward: -1.000, steps: 50
Episode 24: reward: -1.000, steps: 47
Episode 25: reward: 1.000, steps: 50
Episode 26: reward: -1.000, steps: 36
Epis